In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
def parser_image_preds(single_image):
    image_preds = ""
    prediction = single_image.split("seconds.")[1]
    individual_preds_in_image = prediction.split("wheathead")[1:]
    
    for i in individual_preds_in_image:
        pred = i.split(":")[1]
        conf = float("".join(pred.split('\t')[0]).strip().replace("%",""))/100
        
        ny = "".join(i.split(":")[2:]).replace(")","").split(" ")
        bbox = []
        for inner in ny:
            if inner.lstrip('-').isdigit():
                bbox.append(int(inner))
        
        string_to_add = str(str(conf) + " " + str(bbox).replace("[", "").replace("]", "").replace(",", ""))
        image_preds += " " + string_to_add
    return image_preds


## Notebook compiles darknet from scratch, uses weights that I have trained on my machine, and creates submission file using yolov4 on darknet.

In [2]:
cp -r /kaggle/input/darknet-jpo2/* ./

In [3]:
os.chdir("/kaggle/working/darknet_jpo/darknet/")

In [4]:
!ls 

3rdparty		cfg		       libdarknet.so
CMakeLists.txt		cmake		       net_cam_v3.sh
DarknetConfig.cmake.in	darknet		       obj
LICENSE			darknet.py	       predictions.jpg
Makefile		darknet_video.py       results
README.md		data		       scripts
backup			image_yolov2.sh        src
build			image_yolov3.sh        uselib
build.ps1		include		       video_v2.sh
build.sh		json_mjpeg_streams.sh  video_yolov3.sh


In [5]:
!cp ../../yolov4-wheat100_best_v2.weights backup/wheat100/

In [6]:
!ls backup/wheat100/

yolov4-wheat100_best.weights  yolov4-wheat100_best_v2.weights


In [7]:
!sed '1 s/^.*$/GPU=0/; 2 s/^.*$/CUDNN=0/; 3 s/^.*$/CUDNN_HALF=0/; 5 s/^.*$/AVX=0/; 6 s/^.*$/OPENMP=0/; 7 s/^.*$/LIBSO=0/' -i Makefile

In [8]:
!head Makefile

GPU=0
CUDNN=0
CUDNN_HALF=0
OPENCV=1
AVX=0
OPENMP=0
LIBSO=0
ZED_CAMERA=0 # ZED SDK 3.0 and above
ZED_CAMERA_v2_8=0 # ZED SDK 2.X



In [9]:
! make clean

rm -rf ./obj/image_opencv.o ./obj/http_stream.o ./obj/gemm.o ./obj/utils.o ./obj/dark_cuda.o ./obj/convolutional_layer.o ./obj/list.o ./obj/image.o ./obj/activations.o ./obj/im2col.o ./obj/col2im.o ./obj/blas.o ./obj/crop_layer.o ./obj/dropout_layer.o ./obj/maxpool_layer.o ./obj/softmax_layer.o ./obj/data.o ./obj/matrix.o ./obj/network.o ./obj/connected_layer.o ./obj/cost_layer.o ./obj/parser.o ./obj/option_list.o ./obj/darknet.o ./obj/detection_layer.o ./obj/captcha.o ./obj/route_layer.o ./obj/writing.o ./obj/box.o ./obj/nightmare.o ./obj/normalization_layer.o ./obj/avgpool_layer.o ./obj/coco.o ./obj/dice.o ./obj/yolo.o ./obj/detector.o ./obj/layer.o ./obj/compare.o ./obj/classifier.o ./obj/local_layer.o ./obj/swag.o ./obj/shortcut_layer.o ./obj/activation_layer.o ./obj/rnn_layer.o ./obj/gru_layer.o ./obj/rnn.o ./obj/rnn_vid.o ./obj/crnn_layer.o ./obj/demo.o ./obj/tag.o ./obj/cifar.o ./obj/go.o ./obj/batchnorm_layer.o ./obj/art.o ./obj/region_layer.o ./obj/reorg_layer.o ./obj/reorg_ol

In [10]:
!make 

chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:910:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void cv_draw_object(image, float*, int, int, int*, float*, int*, int, char**)’:
./src/image_opencv.cpp:1391:14: warning: unused variable ‘buff’ [-Wunused-variable]
         char buff[100];
              ^~~~
./src/image_opencv.cpp:1367:9: warning: unused variable ‘it_tb_res’ [-Wunused-variable]
     int it_tb_res = cv::createTrackbar(it_trackbar_name, window_name, &it_trackbar_value, 1000);

### Get list of test ids

In [11]:
test_subs = pd.read_csv("/kaggle/input/global-wheat-detection/sample_submission.csv")
test_image_ids = test_subs.image_id.values
test_subs.head()

,image_id,PredictionString
0,aac893a91,1.0 0 0 50 50
1,51f1be19e,1.0 0 0 50 50
2,f5a1f0358,1.0 0 0 50 50
3,796707dd7,1.0 0 0 50 50
4,51b3e36ab,1.0 0 0 50 50


In [12]:
test_image_ids

array(['aac893a91', '51f1be19e', 'f5a1f0358', '796707dd7', '51b3e36ab',
       '348a992bb', 'cc3532ff6', '2fd875eaa', 'cb8d261a3', '53f253011'],
      dtype=object)

In [13]:
from pathlib import Path
import subprocess

root_image = Path("/kaggle/input/global-wheat-detection/test")
test_images = [root_image / f"{img}.jpg" for img in test_subs.image_id]

In [14]:
with open("test_predict.txt", 'a') as file: # append lines to file
    for i in test_image_ids:
        new_name =  "/kaggle/input/global-wheat-detection/test/{}.jpg".format(i)
        new_name = new_name + "\n"
        file.write(new_name)

In [15]:
cmd = "./darknet detector test data/wheat100.data cfg/yolov4-wheat100.cfg backup/wheat100/yolov4-wheat100_best_v2.weights -dont_show -ext_output < test_predict.txt > pr_res.txt -thresh 0.25"
b = subprocess.getoutput(cmd)

In [16]:
!head pr_res.txt

 GPU isn't used 
mini_batch = 1, batch = 1, time_steps = 1, train = 0 
nms_kind: greedynms (1), beta = 0.600000 
nms_kind: greedynms (1), beta = 0.600000 
nms_kind: greedynms (1), beta = 0.600000 

 seen 64, trained: 155 K-images (2 Kilo-batches_64) 
Enter Image Path: /kaggle/input/global-wheat-detection/test/aac893a91.jpg: Predicted in 35871.303000 milli-seconds.
wheathead: 27%	(left_x:   -1   top_y:  909   width:   15   height:   53)
wheathead: 96%	(left_x:   32   top_y:  455   width:   92   height:  152)


In [17]:
conf_result_file = "pr_res.txt"
file_l = open(conf_result_file, 'r')   
lines = file_l.read().split('\n')
content_str = "".join(lines)
int_unique_predictions = content_str.split('Enter')[1:]

In [18]:
res_lines = []  
#single_image = int_unique_predictions[0]
for single_image in int_unique_predictions[:-1]:
    img_id = "".join([i for i in single_image.split(" ") if '.jpg' in i]).replace(":","").split("/")[-1].replace(".jpg", "")
    print(img_id)
    xx = parser_image_preds(single_image)
    #print(img_id, xx)
    res_lines.append({'image_id': img_id, 'PredictionString': xx})

submission_df = pd.DataFrame(res_lines)     

aac893a91
51f1be19e
f5a1f0358
796707dd7
51b3e36ab
348a992bb
cc3532ff6
2fd875eaa
cb8d261a3
53f253011


In [19]:
submission_df.head()

,image_id,PredictionString
0,aac893a91,0.27 -1 909 15 53 0.96 32 455 92 152 0.85 70 ...
1,51f1be19e,0.34 -2 386 52 169 0.92 25 1 92 70 0.83 74 69...
2,f5a1f0358,0.92 -2 3 71 60 0.94 -1 844 67 111 0.95 60 46...
3,796707dd7,0.87 -2 456 69 69 0.31 0 804 61 146 0.53 29 1...
4,51b3e36ab,0.94 -3 431 97 282 0.9 -2 911 84 107 0.64 -2 ...


In [20]:
os.chdir("/kaggle/working/")

In [21]:
submission_df.to_csv('submission.csv', index=False)

In [22]:
!ls

__notebook__.ipynb  submission.csv
darknet_jpo	    yolov4-wheat100_best_v2.weights


In [23]:
!rm -r darknet

rm: cannot remove 'darknet': No such file or directory
